# Particle Identification Task Using Neural Networks

In this project, I will train a classifier to identify type of a particle. There are six particle types: electron, proton, muon, kaon, pion and ghost. Ghost is a particle with other type than the first five or a detector noise. 

Different particle types remain different responses in the detector systems or subdetectors. Thre are five systems: tracking system, ring imaging Cherenkov detector (RICH), electromagnetic and hadron calorimeters, and muon system.

![pid](Images/pid.jpg)

My aim is to identify a particle type using the responses in the detector systems. 

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas
import numpy
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
import utils

# Read Data File

In [ ]:
data = pandas.read_csv('Data/training.csv.gz')

In [ ]:
data.head()

### Domain Information

Following quantities stands for
+ **Spd** : Scintillating Pad Detector
+ **Prs** : Preshower
+ **Ecal** : Electromagnetic Calorimeter
+ **Hcal** : Hadronic Calorimeter
+ **Brem** : Denotes traces of the Particles that were deflected by Detector.

Column Descripions are as follows:

- ***ID*** : id value for tracks (presents only in the test file for the submitting purposes)
- ***Label*** : string valued observable denoting particle types. Can take values "Electron", "Muon", "Kaon", "Proton", "Pion" and "Ghost". This column is absent in the test file.
- ***FlagSpd*** : flag (0 or 1), if reconstructed track passes through Spd
- ***FlagPrs*** : flag (0 or 1), if reconstructed track passes through Prs
- ***FlagBrem*** : flag (0 or 1), if reconstructed track passes through Brem
- ***FlagEcal*** : flag (0 or 1), if reconstructed track passes through Ecal
- ***FlagHcal*** : flag (0 or 1), if reconstructed track passes through Hcal
- ***FlagRICH1*** : flag (0 or 1), if reconstructed track passes through the first RICH detector
- ***FlagRICH2*** : flag (0 or 1), if reconstructed track passes through the second RICH detector
- ***FlagMuon*** : flag (0 or 1), if reconstructed track passes through muon stations (Muon)
- ***SpdE*** : energy deposit associated to the track in the Spd
- ***PrsE*** : energy deposit associated to the track in the Prs
- ***EcalE*** : energy deposit associated to the track in the Hcal
- ***HcalE*** : energy deposit associated to the track in the Hcal
- ***PrsDLLbeElectron*** : delta log-likelihood for a particle candidate to be electron using information from Prs
- ***BremDLLbeElectron*** : delta log-likelihood for a particle candidate to be electron using information from Brem
- ***TrackP*** : particle momentum
- ***TrackPt*** : particle transverse momentum
- ***TrackNDoFSubdetector1*** : number of degrees of freedom for track fit using hits in the tracking sub-detector1
- ***TrackQualitySubdetector1*** : chi2 quality of the track fit using hits in the tracking sub-detector1
- ***TrackNDoFSubdetector2*** : number of degrees of freedom for track fit using hits in the tracking sub-detector2
- ***TrackQualitySubdetector2*** : chi2 quality of the track fit using hits in the  tracking sub-detector2
- ***TrackNDoF*** : number of degrees of freedom for track fit using hits in all tracking sub-detectors
- ***TrackQualityPerNDoF*** : chi2 quality of the track fit per degree of freedom
- ***TrackDistanceToZ*** : distance between track and z-axis (beam axis)
- ***Calo2dFitQuality*** : quality of the 2d fit of the clusters in the calorimeter 
- ***Calo3dFitQuality*** : quality of the 3d fit in the calorimeter with assumption that particle was electron
- ***EcalDLLbeElectron*** : delta log-likelihood for a particle candidate to be electron using information from Ecal
- ***EcalDLLbeMuon*** : delta log-likelihood for a particle candidate to be muon using information from Ecal
- ***EcalShowerLongitudinalParameter*** : longitudinal parameter of Ecal shower
- ***HcalDLLbeElectron*** : delta log-likelihood for a particle candidate to be electron using information from Hcal
- ***HcalDLLbeMuon*** : delta log-likelihood for a particle candidate to be using information from Hcal
- ***RICHpFlagElectron*** : flag (0 or 1) if momentum is greater than threshold for electrons to produce Cherenkov light
- ***RICHpFlagProton*** : flag (0 or 1) if momentum is greater than threshold for protons to produce Cherenkov light
- ***RICHpFlagPion*** : flag (0 or 1) if momentum is greater than threshold for pions to produce Cherenkov light
- ***RICHpFlagKaon*** : flag (0 or 1) if momentum is greater than threshold for kaons to produce Cherenkov light
- ***RICHpFlagMuon*** : flag (0 or 1) if momentum is greater than threshold for muons to produce Cherenkov light
- ***RICH_DLLbeBCK *** : delta log-likelihood for a particle candidate to be background using information from RICH
- ***RICH_DLLbeKaon*** : delta log-likelihood for a particle candidate to be kaon using information from RICH
- ***RICH_DLLbeElectron*** : delta log-likelihood for a particle candidate to be electron using information from RICH
- ***RICH_DLLbeMuon*** : delta log-likelihood for a particle candidate to be muon using information from RICH
- ***RICH_DLLbeProton*** : delta log-likelihood for a particle candidate to be proton using information from RICH
- ***MuonFlag*** : muon flag (is this track muon) which is determined from muon stations
- ***MuonLooseFlag*** : muon flag (is this track muon) which is determined from muon stations using looser criteria
- ***MuonLLbeBCK*** : log-likelihood for a particle candidate to be not muon using information from muon stations
- ***MuonLLbeMuon*** : log-likelihood for a particle candidate to be muon using information from muon stations
- ***DLLelectron*** : delta log-likelihood for a particle candidate to be electron using information from all subdetectors
- ***DLLmuon*** : delta log-likelihood for a particle candidate to be muon using information from all subdetectors
- ***DLLkaon*** : delta log-likelihood for a particle candidate to be kaon using information from all subdetectors
- ***DLLproton*** : delta log-likelihood for a particle candidate to be proton using information from all subdetectors
- ***GhostProbability*** : probability for a particle candidate to be ghost track. This variable is an output of classification model used in the tracking algorithm.

Delta log-likelihood in the features descriptions means the difference between log-likelihood for the mass hypothesis that a given track is left by some particle (for example, electron) and log-likelihood for the mass hypothesis that a given track is left by a pion (so, DLLpion = 0 and thus we don't have these columns). This is done since most tracks (~80%) are left by pions and in practice we actually need to discriminate other particles from pions. In other words, the null hypothesis is that particle is a pion.

In [ ]:
# Classification Labels

set(data.Label)

In [ ]:
# Converting Labels into Numerical Factor

data['Class'] = utils.get_class_ids(data.Label.values)
set(data.Class)

### Training Features

The following set of features describe particle responses in the detector systems:

![features](Images/features.jpeg)

Also there are several combined features. The full list is following.

In [ ]:
features = list(set(data.columns) - {'Label', 'Class'})
features

### Feature Engineering

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from itertools import combinations

In [ ]:
# Adding New Features

def add_features(data):
    trans = MinMaxScaler()
    data["TRACK"] = (1e-20 + trans.fit_transform(data[[x for x in data.columns if "track" in x.lower()]])).prod(axis=1)
    data["RICH"] = (1e-20 + trans.fit_transform(data[[x for x in data.columns if "rich" in x.lower()]])).prod(axis=1)
    data["ELECTRO"] = (1e-20 + trans.fit_transform(data[[x for x in data.columns if "ecal" in x.lower()]])).prod(axis=1)
    data["HADRON"] = (1e-20 + trans.fit_transform(data[[x for x in data.columns if "hcal" in x.lower()]])).prod(axis=1)
    data["MUON"] = (1e-20 + trans.fit_transform(data[[x for x in data.columns if "muon" in x.lower() and 
                                                                                 "rich" not in x.lower() and 
                                                                                 "ecal" not in x.lower()]])).prod(axis=1)
    
    for a,b in combinations(["TRACK", "RICH", "ELECTRO", "HADRON", "MUON"], 2):
        data["{}__X__{}".format(a.lower(), b.lower())] = data[a] * data[b]

    mask_df = 1 * (data[ data.columns[data.min() == -999] ] != -999)
    mask_df.columns = [x+"_mask" for x in mask_df.columns]

    square_df = 0
    if "Label" in data.columns:
        square_df = data.drop("Label", axis=1) **2
    else:
        square_df = data**2
    square_df.columns = [x+"_sq" for x in square_df.columns]

    data = pandas.concat([data, mask_df, square_df], axis=1)

    return data

In [ ]:
# Update the Features list

features = list(set(data.columns) - {'Label', 'Class'})

# Data Splits

In [ ]:
training_data, validation_data = train_test_split(data, random_state=11, train_size=0.90)

In [ ]:
len(training_data), len(validation_data)

# Neural Network Classifier

In [ ]:
from keras.layers.core import Dense, Activation, Dropout, Reshape, Input
from keras.layers.advanced_activations import LeakyReLU, PReLU, ReLU, Softmax
from keras.layers.normalization import BatchNormalization
from keras.layers import Input, Embedding, multiply
import keras.initializers as inits
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras.utils import np_utils

In [ ]:
def classifier(input_dim):
    model = Sequential()
    model.add(Dense(100, input_dim=input_dim, kernel_initializer=inits.glorot_normal()))
    model.add(BatchNormalization())
    model.add(ReLU())
    
    model.add(Dense(100, input_dim=input_dim, kernel_initializer=inits.glorot_normal()))
    model.add(BatchNormalization())
    model.add(ReLU())

    model.add(Dense(6, kernel_initializer=inits.glorot_normal()))
    model.add(Softmax())

    model.compile(loss='categorical_crossentropy', optimizer=Adam())
    return model

### Data Preprocessing

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
sd_obj = StandardScaler()
sd_obj.fit(training_data[features].values)

X_train = sd_obj.transform(training_data[features].values)

### Training

In [ ]:
nn = classifier(X.shape[1])

nn.fit(X_train, 
      np_utils.to_categorical(training_data.Class.values), 
      verbose=1, 
      epochs=10, 
      batch_size=256)

### Log Loss on the Validation sample

In [ ]:
X_valid = sd_obj.transform(validation_data[features].values)

In [ ]:
# Prediction for each Track [Aim: Hit lower than or equal to 0.525]

proba_nn = nn.predict_proba(X_valid)
log_loss(validation_data.Class.values, proba_nn)

### Quality Metrics

Plotting ROC curves and signal efficiency dependece from particle mometum and transverse momentum values.

In [ ]:
proba = proba_nn

In [ ]:
utils.plot_roc_curves(proba, validation_data.Class.values)

In [ ]:
utils.plot_signal_efficiency_on_p(proba, validation_data.Class.values, validation_data.TrackP.values, 60, 50)
plt.show()

In [ ]:
utils.plot_signal_efficiency_on_pt(proba, validation_data.Class.values, validation_data.TrackPt.values, 60, 50)
plt.show()

# Decorrelation Using Adversarial Neural Network